# Reconstruction: linear 

The spatial reconstruction of the fundamental QNM compared with the corresponding spheroidal harmonic, plotted on the sphere in Mollweide projection. 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import CCE 
from qnmfits.spatial_mapping_functions import *

plt.style.use("stylesheet.mplstyle")
fig_width = 246.0 * (1.0 / 72.27)
fig_height = fig_width / 1.618

In [2]:
sim = CCE.SXS_CCE("0001")

l_max = 8
n_max = 7

t0 = 17.0
mapping = [(2, 2, 0, 1)]
map = mapping[0]

In [ ]:
QNMs = [
    (lam, 2, n, p)
    for lam in np.arange(2, l_max + 1)
    for n in np.arange(0, n_max + 1)
    for p in (-1, +1)
]

best_fit = mapping_multimode_ringdown_fit(
    sim.times,
    sim.h,
    modes=QNMs,
    Mf=sim.Mf,
    chif=sim.chif_mag,
    t0=t0,
    mapping_modes=mapping,
    spherical_modes=[(l, 2) for l in np.arange(2, l_max + 1)],
)

sm, _, _ = spatial_mismatch_linear(best_fit, map, sim.chif_mag, l_max)
print(sm)

In [ ]:
lon = np.linspace(-np.pi, np.pi, 200)
lat = np.linspace(-np.pi / 2, np.pi / 2, 200)

map = mapping[0]

Lon, Lat = np.meshgrid(lon, lat)

F = spatial_reconstruction(np.pi / 2 - Lat, Lon, best_fit, map, l_max)
G = spatial_prediction_linear(np.pi / 2 - Lat, Lon, map, l_max, sim.chif_mag)

fig, axs = plt.subplots(
    nrows=1,
    ncols=2,
    subplot_kw={"projection": "mollweide"},
    figsize=(fig_width * 2, fig_height * 2),
)

axs[0].title.set_text("Reconstruction")
axs[0].pcolormesh(Lon, Lat, np.real(F), cmap=plt.cm.RdBu)
axs[0].set_xticks(np.radians([-120, -60, 0, 60, 120]))
axs[0].set_yticks(np.radians([-60, -30, 0, 30, 60]))

axs[1].title.set_text("Predicted")
im3 = axs[1].pcolormesh(Lon, Lat, np.real(G), cmap=plt.cm.RdBu)
axs[1].set_xticks(np.radians([-120, -60, 0, 60, 120]))
axs[1].set_yticks(np.radians([-60, -30, 0, 30, 60]))

cax = fig.add_axes([0.25, 0.25, 0.51, 0.04])
fig.colorbar(im3, cax=cax, orientation="horizontal")
fig.text(0.5, 0.31, r"$\mathrm{Re}(A_{(220+)}(\theta , \phi))$", ha="center")

axs[0].tick_params(which="both")
axs[1].tick_params(which="both")

fig.savefig("figs/linear_mapping_reconstruction.png", dpi=600, bbox_inches="tight")
plt.show()